In [ ]:
import importlib
import sys
import torch
import pickle
import os

sys.path.insert(0, '..')
sys.path.insert(0, '../..')
sys.path.insert(0, '../../..')
sys.path.insert(0, '../../../..')
sys.path.insert(0, '../../../../..')
sys.path.insert(0, '../../../../../..')


from model.dropout_uncertainty_enc_dec_LSTM.dropout_uncertainty_model import DropoutUncertaintyEncoderDecoderLSTM

In [ ]:
#load model
file_path_model = '../../training_variational_dropout/BPIC17/BPIC_2017_full_grad_norm_new_4layer.pkl'

model = DropoutUncertaintyEncoderDecoderLSTM.load(file_path_model, dropout=0.1)

# Load the dataset
file_path_data_set = '../../../../encoded_data/BPIC_2017_all_5_test.pkl'
bpic_17_test_dataset = torch.load(file_path_data_set, weights_only=False)

In [ ]:
import src.evaluation.probabilistic_evaluation
importlib.reload(src.evaluation.probabilistic_evaluation)
from src.evaluation.probabilistic_evaluation import ProbabilisticEvaluation

new_eval = ProbabilisticEvaluation(model, bpic_17_test_dataset,
                                   num_processes=32,
                                   growing_num_values = ['case_elapsed_time'],
                                   samples_per_case = 1000,
                                   sample_argmax = False,
                                   use_variance_cat = True,
                                   use_variance_num = True,
                                   all_cat= ['concept:name', 'org:resource', 'lifecycle:transition'],
                                   all_num= ['case_elapsed_time', 'event_elapsed_time']
                                   )

In [ ]:
def save_chunk(results, i):
    chunk_number = (i + 1)
    filename = os.path.join(output_dir, f'results_part_{chunk_number:03d}.pkl')
    with open(filename, 'wb') as f:
        pickle.dump(results, f)
    print(f"Saved {len(results)} results to {filename}")

output_dir = '../../../../../../evaluation_results/BPIC17/gn'
save_every = 500
results = {}
for i, (case_name, prefix_len, prefix, predicted_suffixes, suffix, mean_prediction) in enumerate(new_eval.evaluate_multi_processing(random_order=False, include_model_states=False)):
    print(case_name, prefix_len)
    assert((case_name, prefix_len) not in results)
    results[(case_name, prefix_len)] = (prefix, suffix, mean_prediction, predicted_suffixes)
    print(prefix_len, len(suffix))
    if (i + 1) % save_every == 0:
        save_chunk(results, i)
        results = {}

if len(results):
    save_chunk(results, i)